## Problem Statement

### Context

AllLife Bank is a US bank that has a growing customer base. The majority of these customers are liability customers (depositors) with varying sizes of deposits. The number of customers who are also borrowers (asset customers) is quite small, and the bank is interested in expanding this base rapidly to bring in more loan business and in the process, earn more through the interest on loans. In particular, the management wants to explore ways of converting its liability customers to personal loan customers (while retaining them as depositors).

A campaign that the bank ran last year for liability customers showed a healthy conversion rate of over 9% success. This has encouraged the retail marketing department to devise campaigns with better target marketing to increase the success ratio.

You as a Data scientist at AllLife bank have to build a model that will help the marketing department to identify the potential customers who have a higher probability of purchasing the loan.

### Objective

To predict whether a liability customer will buy personal loans, to understand which customer attributes are most significant in driving purchases, and identify which segment of customers to target more.

### Data Dictionary
* `ID`: Customer ID
* `Age`: Customer’s age in completed years
* `Experience`: #years of professional experience
* `Income`: Annual income of the customer (in thousand dollars)
* `ZIP Code`: Home Address ZIP code.
* `Family`: the Family size of the customer
* `CCAvg`: Average spending on credit cards per month (in thousand dollars)
* `Education`: Education Level. 1: Undergrad; 2: Graduate;3: Advanced/Professional
* `Mortgage`: Value of house mortgage if any. (in thousand dollars)
* `Personal_Loan`: Did this customer accept the personal loan offered in the last campaign? (0: No, 1: Yes)
* `Securities_Account`: Does the customer have securities account with the bank? (0: No, 1: Yes)
* `CD_Account`: Does the customer have a certificate of deposit (CD) account with the bank? (0: No, 1: Yes)
* `Online`: Do customers use internet banking facilities? (0: No, 1: Yes)
* `CreditCard`: Does the customer use a credit card issued by any other Bank (excluding All life Bank)? (0: No, 1: Yes)

## Importing necessary libraries

In [ ]:
# Installing the libraries with the specified version.
!pip install numpy==1.25.2 pandas==1.5.3 matplotlib==3.7.1 seaborn==0.13.1 scikit-learn==1.2.2 sklearn-pandas==2.2.0 uszipcode==1.0.1 -q --user

**Note**:

1. After running the above cell, kindly restart the notebook kernel (for Jupyter Notebook) or runtime (for Google Colab), write the relevant code for the project from the next cell, and run all cells sequentially from the next cell.

2. On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

# import all of required libraries and dependecies setting in next code block

In [1]:
# Library to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Data manipulation Libraries
import pandas as pd
import numpy as np

# Library to split data for testing and training model
from sklearn.model_selection import train_test_split

# Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Display as many columns in data
pd.set_option("display.max_columns", None)
# Limit number of rows to be displayed to 100 to avoid long list
pd.set_option("display.max_rows", 100)

# Logistic Regression Library
from sklearn.linear_model import LogisticRegression

# Decision Tree Classifier Libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# Import Grid Search Model for Exhaustive search over specified parameter values for an estimator.
from sklearn.model_selection import GridSearchCV

# import specific modules from sklearn metrics library
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    roc_curve,
    roc_auc_score,
    make_scorer,

)

# import the SearchEngine from uszipcode library
# Note - latest uszipcode==1.0.1 has incompatiblity with latest SQLAlchemy and sqlalchemy_mate
# You may need to install dependecies in this sequence to ensure uszipcode library works without error
# pip install sqlalchemy_mate==1.4.28.4
# pip install SQLAlchemy==1.4.32
# pip install uszipcode==1.0.1
# pip install python-Levenshtein

from uszipcode import SearchEngine


## Loading the dataset

In [8]:
# Load data file
data = pd.read_csv("Loan_Modelling.csv")
# Make a copy of the data to avoid any changes to the original data during manipulation
loan = data.copy()

## Data Overview

* Observations
* Sanity checks

In [9]:
# Display 10 random sample rows of the dataset
loan.sample(10)

,ID,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
1997,1998,54,30,61,92093,1,1.8,3,0,0,0,0,1,0
4978,4979,57,27,63,90210,4,2.0,3,0,0,0,0,1,0
1855,1856,65,39,30,94304,3,0.7,2,0,0,0,0,1,1
2701,2702,50,26,55,94305,1,1.6,2,0,0,0,0,1,0
1701,1702,29,3,108,94304,4,1.8,2,0,0,0,0,0,0
1553,1554,46,22,83,95616,3,0.7,1,0,0,0,0,0,0
460,461,60,36,141,90277,2,2.1,1,0,0,0,0,1,1
2562,2563,45,21,39,92521,2,2.1,3,184,0,0,0,0,1
452,453,39,13,21,94022,3,0.2,2,0,0,0,0,0,0
1140,1141,32,6,13,94611,4,0.3,1,0,0,0,0,1,1


In [10]:
# Lets drop the ID Column from dataset, as it does not add specific predictibility value
loan.drop("ID", axis=1, inplace=True)
loan.sample(10)

,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard
945,57,32,33,91745,1,1.50,2,0,0,0,0,1,0
3318,46,20,105,90089,4,3.20,1,0,1,0,0,0,0
1191,29,5,128,94111,1,1.50,1,0,0,0,0,1,1
4670,52,26,194,94305,1,1.70,1,0,0,0,0,1,0
4807,40,14,53,90064,1,2.00,1,0,0,0,0,1,1
2116,44,17,70,94920,3,2.67,2,0,0,0,0,0,0
1349,26,2,171,93943,3,6.00,2,0,1,0,0,1,0
1187,61,36,24,94309,1,1.50,2,87,0,1,0,0,0
1387,35,10,38,95762,4,1.70,1,0,0,0,0,1,1
3960,62,37,48,92028,3,2.20,1,0,0,0,0,0,0


In [11]:
# print first and last 5 rows of the dataset.
print(loan.head())
print("-" * 75)
print(loan.tail())


   Age  Experience  Income  ZIPCode  Family  CCAvg  Education  Mortgage  \
0   25           1      49    91107       4    1.6          1         0   
1   45          19      34    90089       3    1.5          1         0   
2   39          15      11    94720       1    1.0          1         0   
3   35           9     100    94112       1    2.7          2         0   
4   35           8      45    91330       4    1.0          2         0   

   Personal_Loan  Securities_Account  CD_Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  
---------------------------------------------------------------------------
      Age  Experience  Income  ZIPCode  Family 

In [12]:
# Check for null values
loan.isnull().sum()


Age                   0
Experience            0
Income                0
ZIPCode               0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal_Loan         0
Securities_Account    0
CD_Account            0
Online                0
CreditCard            0
dtype: int64

In [13]:
# Check for duplicate values
loan.duplicated().sum()

0

# Observation: 
 - No duplicate values found in the data set
 - No null values found in the data set

In [14]:
# check shape of the data set
print("Dataset shape: ", loan.shape)

Dataset shape:  (5000, 13)


In [15]:
# check variables datatypes
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 5000 non-null   int64  
 1   Experience          5000 non-null   int64  
 2   Income              5000 non-null   int64  
 3   ZIPCode             5000 non-null   int64  
 4   Family              5000 non-null   int64  
 5   CCAvg               5000 non-null   float64
 6   Education           5000 non-null   int64  
 7   Mortgage            5000 non-null   int64  
 8   Personal_Loan       5000 non-null   int64  
 9   Securities_Account  5000 non-null   int64  
 10  CD_Account          5000 non-null   int64  
 11  Online              5000 non-null   int64  
 12  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(12)
memory usage: 507.9 KB


In [16]:
# Data Summary
loan.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,5000.0,45.338400,11.463166,23.0,35.0,45.0,55.0,67.0
Experience,5000.0,20.104600,11.467954,-3.0,10.0,20.0,30.0,43.0
Income,5000.0,73.774200,46.033729,8.0,39.0,64.0,98.0,224.0
ZIPCode,5000.0,93169.257000,1759.455086,90005.0,91911.0,93437.0,94608.0,96651.0
Family,5000.0,2.396400,1.147663,1.0,1.0,2.0,3.0,4.0
CCAvg,5000.0,1.937938,1.747659,0.0,0.7,1.5,2.5,10.0
Education,5000.0,1.881000,0.839869,1.0,1.0,2.0,3.0,3.0
Mortgage,5000.0,56.498800,101.713802,0.0,0.0,0.0,101.0,635.0
Personal_Loan,5000.0,0.096000,0.294621,0.0,0.0,0.0,0.0,1.0
Securities_Account,5000.0,0.104400,0.305809,0.0,0.0,0.0,0.0,1.0


## Observations (in order of display):
Age - min Age is 23 , max is 67 , Average is 45 Years old - Data seems normal
Experience - min Experience is -3 years, looks odd
Income - min Income is USD 46k, max is USD 224K and average is USD 64K - Data Appears to be right skewed.
ZIPCode - Zip codes should be converted to City/State using uszip library to avoid treating it as plain number as its location.
Family - min Family size is 1, max is 4 and average is 2 - Data looks reasonable and normally distributed, we can use it as category.
CCAvg - min CCAvg is USD 0k, max is USD 1.9K and average is USD 10K - Data appears normal but right skewed.
Education -  Education is a categorical variable for 1 as Undergrad, 2 as Graduate and 3 as Professional
Mortgage - min Mortgage is USD 0, max is USD 635K and average is USD 0 - Data appears right skewed
Personal_Loan - Personal_Loan is a categorical variable where 1: customer accepted the personal loan offered in the last campaign and 0:customer didnot accept
Securities_Account - Securities_Account is a categorical variable where 1: customer has Securities_Account 0:customer does not have Securities_Account
CD_Account - CD_Account is a categorical variable where 1: customer has CD_Account 0:customer does not have CD_Account
Online - Online is a categorical variable where 1: customer uses online banking 0:customer does use online banking
CreditCard - CreditCard is a categorical variable where 1: customer use a credit card issued by any other Bank 0:customer does not use a credit card issued by any other Bank

### ZIPCode coversion to City and State 

In [17]:
# Find City and State for each ZIP code
# Create the search engine fro uszipcode library
search = SearchEngine()

# Function to extract the city using zipcode value
def get_city(x):
    return search.by_zipcode(x).city

# Function to extract state using zipcode value
def get_state(x):
    return search.by_zipcode(x).state

Download D:\Dev\Home\.uszipcode\simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


In [18]:
# Now populate city and state for each record/zip code
city_list = []
state_list =[]

# Loop through 5000 records in data, to extract city/state and add value to corrosponding lists
for i in np.arange(0,5000):
    try:
        city = get_city(loan["ZIPCode"].iloc[i])
        city_list.append(city)
        state = get_state(loan["ZIPCode"].iloc[i])
        state_list.append(state)
    except:
        city_list.append(np.nan)
        state_list.append(np.nan)
    continue

In [47]:
# Add new columns, city and state from lists in the dataset
loan['City'] = city_list
loan['State'] = state_list
loan.head(10)

,Age,Experience,Income,ZIPCode,Family,CCAvg,Education,Mortgage,Personal_Loan,Securities_Account,CD_Account,Online,CreditCard,City,State
0,25,1,49,91107,4,1.6,1,0,0,1,0,0,0,Pasadena,CA
1,45,19,34,90089,3,1.5,1,0,0,1,0,0,0,Los Angeles,CA
2,39,15,11,94720,1,1.0,1,0,0,0,0,0,0,Berkeley,CA
3,35,9,100,94112,1,2.7,2,0,0,0,0,0,0,San Francisco,CA
4,35,8,45,91330,4,1.0,2,0,0,0,0,0,1,Northridge,CA
5,37,13,29,92121,4,0.4,2,155,0,0,0,1,0,San Diego,CA
6,53,27,72,91711,2,1.5,2,0,0,0,0,1,0,Claremont,CA
7,50,24,22,93943,1,0.3,3,0,0,0,0,0,1,Monterey,CA
8,35,10,81,90089,3,0.6,2,104,0,0,0,1,0,Los Angeles,CA
9,34,9,180,93023,1,8.9,3,0,1,0,0,0,0,Ojai,CA


In [48]:
# Check if any values in City column is null
loan['City'].isnull().value_counts()

City
False    4966
True       34
Name: count, dtype: int64

In [49]:
# Check if any values in State column is null
loan['State'].isnull().value_counts()

State
False    4966
True       34
Name: count, dtype: int64

In [50]:
# Find out which zipcodes did not get city data from uszip library
zip_city_nan = loan[loan['City'].isnull()]
zip_city_nan['ZIPCode'].value_counts()

ZIPCode
92717    22
96651     6
92634     5
93077     1
Name: count, dtype: int64

In [51]:
# Find out which zipcodes did not get state data from uszip library
zip_state_nan = loan[loan['State'].isnull()]
zip_state_nan['ZIPCode'].value_counts()

ZIPCode
92717    22
96651     6
92634     5
93077     1
Name: count, dtype: int64

### Missing values for City and State is for same set of 4 zip codes, most likely uszip library is missing some newer zip codes due to stale data

In [52]:
# Google Search missing Zip codes to manually populate City/State Data, 96651 belongs to United States Armed Forces (Naval/Marine)
missing_zip_values = {'92717':'Irvine, CA',
             '96651':'Naval-Marine, AP',
             '92634':'Fullerton, CA',
             '93077':'Ventura, CA'
            }

# Function to fill the missing values into dataset
def fill_nan(data, indxs, value, column):
    for i in indxs:
        data[column].iloc[i]=value
        
# Loop through missing_zip_values to fill in the missing city and state values    
for i in missing_zip_values.keys():
    indxs = loan[loan['ZIPCode']==int(i)].index
    fill_nan(loan, indxs, missing_zip_values[str(i)].split(',')[0], 'City')
    fill_nan(loan, indxs, missing_zip_values[str(i)].split(',')[1], 'State')

# Check if City/State null values removed from dataset
loan.isnull().sum()

Age                   0
Experience            0
Income                0
ZIPCode               0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal_Loan         0
Securities_Account    0
CD_Account            0
Online                0
CreditCard            0
City                  0
State                 0
dtype: int64

In [53]:
# Strip any whitespaces from newly populated columns
loan['City']=loan['City'].str.strip()
loan['State']=loan['State'].str.strip()

# Show statistics of City/State
print(f'{loan['City'].value_counts()}')
print("-" * 40)
print(f'{loan['State'].value_counts()}')

City
Los Angeles      375
San Diego        269
San Francisco    257
Berkeley         241
Sacramento       148
                ... 
Sausalito          1
Ladera Ranch       1
Sierra Madre       1
Tahoe City         1
Stinson Beach      1
Name: count, Length: 245, dtype: int64
----------------------------------------
State
CA    4994
AP       6
Name: count, dtype: int64


In [54]:
# Show number of data rows having negative value for Experience Column
print(f'There are total {len(loan[loan["Experience"] < 0])} rows with a negative value for Experience')
# Checking on experience < 0
loan[loan["Experience"] < 0]["Experience"].value_counts()

There are total 0 rows with a negative value for Experience


Series([], Name: count, dtype: int64)

In [55]:
# negative values for Experience are likely input errors and we can simply remove -ve sigh from Data 
loan["Experience"].replace(-1, 1, inplace=True)
loan["Experience"].replace(-2, 2, inplace=True)
loan["Experience"].replace(-3, 3, inplace=True)

In [57]:
# Find out that no more -ve values exists for Experience Column
len(loan[loan["Experience"] < 0])

0

In [58]:
# Categorical variables should be converted to data type of categorical variables from numeric.
category_columns = [
    "ZIPCode",
    "Education",
    "Personal_Loan",
    "Securities_Account",
    "CD_Account",
    "Online",
    "CreditCard",
    'City',
    'State'
]
loan[category_columns] = loan[category_columns].astype("category")

In [59]:
# Verify that data types changed properly
loan.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Age                 5000 non-null   int64   
 1   Experience          5000 non-null   int64   
 2   Income              5000 non-null   int64   
 3   ZIPCode             5000 non-null   category
 4   Family              5000 non-null   int64   
 5   CCAvg               5000 non-null   float64 
 6   Education           5000 non-null   category
 7   Mortgage            5000 non-null   int64   
 8   Personal_Loan       5000 non-null   category
 9   Securities_Account  5000 non-null   category
 10  CD_Account          5000 non-null   category
 11  Online              5000 non-null   category
 12  CreditCard          5000 non-null   category
 13  City                5000 non-null   category
 14  State               5000 non-null   category
dtypes: category(9), float64(1), int64(5)
m

## Exploratory Data Analysis.

- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

**Questions**:

1. What is the distribution of mortgage attribute? Are there any noticeable patterns or outliers in the distribution?
2. How many customers have credit cards?
3. What are the attributes that have a strong correlation with the target attribute (personal loan)?
4. How does a customer's interest in purchasing a loan vary with their age?
5. How does a customer's interest in purchasing a loan vary with their education?

## Data Preprocessing

* Missing value treatment
* Feature engineering (if needed)
* Outlier detection and treatment (if needed)
* Preparing data for modeling
* Any other preprocessing steps (if needed)

## Model Building

### Model Evaluation Criterion

*


### Model Building

## Model Performance Improvement

## Model Performance Comparison and Final Model Selection

## Actionable Insights and Business Recommendations


* What recommedations would you suggest to the bank?

___